varient of fine tuning that enabled GPT-3 to turn into ChatGpt.

instruction fine tuning is type of fine tuning. fine tuning tasks - reasoning, routing, chat, agents

instruction fine tuning teaches model to follow instructions.

eg teaches model to behave more like a chatbot

Better user interface for model interaction.

It is the method that turn GPT-3 to ChatGpt

Instruction following datseets :- existing data is ready as-is, online

1. FAQs

2. Customer support conversations

3. Slack messages


LLM Data-Generation 

1. Non-Q&A data can also be converted to Q&A using:-

    1. Prompt template

    2. Using another LLM ChatGPT(Alpaca) or Open-source models



Instruction fine tuning generalization

1. Can access model's pre-existing knowledge

2. Generalize following instructions to other data, not in finetiuning dataset

eg:-

What is capital of France?   

Paris -------Finetuning Data


Can you write a function

that computes the Fibonacci

sequence in python?  --------- Code not in finetuning data, only base data


def fibonacci(n):
    
    sequence = []
    
    for i in range(n):   ---------- model can now answer


Overview of Finetuning

                            Data prep ---> Training ====> Evaluation  ===> Data prep cycle continues. iterative process


Data Prep -> Specifically for instruction finetuning and other different types of finetuning, data prep is really where you have differences.

Here you change your data, tailor your data to specific type of finetuning, 


In [17]:
import os
import itertools
import jsonlines


from dotenv import load_dotenv
from datasets import load_dataset
from pprint import pprint

# from llama import BasicModelRunner
from lamini import Lamini
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer


In [ ]:
load_dotenv()



python-dotenv could not parse statement starting at line 1
python-dotenv could not parse statement starting at line 8


TypeError: unsupported operand type(s) for -: 'str' and 'str'

In [ ]:

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [7]:
instruction_tuned_dataset = load_dataset("tatsu-lab/alpaca", split="train", streaming=True)

In [9]:
m = 5
print("Instruction-tuned dataset:")
top_m = list(itertools.islice(instruction_tuned_dataset, m))

for j in top_m:
    print((j))

Instruction-tuned dataset:


'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: 34b3a69d-1490-4175-a2d2-a1517d897f3f)')' thrown while requesting GET https://huggingface.co/datasets/tatsu-lab/alpaca/resolve/dce01c9b08f87459cf36a430d809084718273017/data/train-00000-of-00001-a09b74b3ef9c3b56.parquet
Retrying in 1s [Retry 1/5].
'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: d5daf2af-1e25-407e-80a4-bd20014d060a)')' thrown while requesting GET https://huggingface.co/datasets/tatsu-lab/alpaca/resolve/dce01c9b08f87459cf36a430d809084718273017/data/train-00000-of-00001-a09b74b3ef9c3b56.parquet
Retrying in 2s [Retry 2/5].
'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: 4bb0b0b8-10f1-4c9c-b9ef-363111576731)')' thrown while requesting GET https://huggingface.co/datasets/tatsu-lab/alpaca/resolve/dce01c9

{'instruction': 'Give three tips for staying healthy.', 'input': '', 'output': '1.Eat a balanced diet and make sure to include plenty of fruits and vegetables. \n2. Exercise regularly to keep your body active and strong. \n3. Get enough sleep and maintain a consistent sleep schedule.', 'text': 'Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\nGive three tips for staying healthy.\n\n### Response:\n1.Eat a balanced diet and make sure to include plenty of fruits and vegetables. \n2. Exercise regularly to keep your body active and strong. \n3. Get enough sleep and maintain a consistent sleep schedule.'}
{'instruction': 'What are the three primary colors?', 'input': '', 'output': 'The three primary colors are red, blue, and yellow.', 'text': 'Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\nWhat are the three primary colors?\n\n### Respo

In [10]:
prompt_template_with_input = """ Below is an instruction that describes the task, paired with an input that provides further context. Write a response that appropriately completes the request.
### Instructions:
{instruction}

### Input:
{input}

### Response: """

prompt_template_without_input = """ Below is an instruction that describe a task. Write a response that appropriately completes the request.

### Instruction:
{instruction}

### Response:"""

In [12]:
processed_data = []

for j in top_m:
    if not j["input"]:
        processed_prompt = prompt_template_without_input.format(instruction = j["instruction"])
    else:
        processed_prompt = prompt_template_with_input.format(intruction = j["instruction"], input = j["input"])
        
    processed_data.append({"input":processed_prompt, "output": j["output"]})

In [13]:
pprint(processed_data[0])

{'input': ' Below is an instruction that describe a task. Write a response '
          'that appropriately completes the request.\n'
          '\n'
          '### Instruction:\n'
          'Give three tips for staying healthy.\n'
          '\n'
          '### Response:',
 'output': '1.Eat a balanced diet and make sure to include plenty of fruits '
           'and vegetables. \n'
           '2. Exercise regularly to keep your body active and strong. \n'
           '3. Get enough sleep and maintain a consistent sleep schedule.'}


In [15]:
with jsonlines.open(os.path.join("/Users/apple/Desktop/project/LLM/Finetuning_llm/Data","alpaca_processed.jsonl"),'w') as writer:
    writer.write_all(processed_data)

In [16]:
# uploading to hugging face
# dataset_pat_hf = "lamini/alpaca"
# dataset_hf = load_dataset(dataset_path_hf)
# print(dataset_hf)

we are done with dataset preprocessing next step is train on different models

In [30]:
non_instruct_model = Lamini(model_name = "meta-llama/Llama-2-7b-hf")
non_instruct_output = non_instruct_model.generate("Tell me how to train my dog to sit")

import re

sentences = re.split(r'(?<=[.!?]) +', non_instruct_output)
print("\n".join(sentences))

# print("Not instruction-tuned ouptput (Llama 2 Base): ",non_instruct_output)


.
Tell me how to train my dog to sit.
I have a 10 month old puppy and I want to train him to sit.
I have tried the treat method and he just sits there and looks at me like I am crazy.
I have tried the "sit" command and he just looks at me like I am crazy.
I have tried the "sit" command and he just looks at me like I am crazy.
I have tried the "sit" command and he just looks at me like I am crazy.
I have tried the "sit" command and he just looks at me like I am crazy.
I have tried the "sit" command and he just looks at me like I am crazy.
I have tried the "sit" command and he just looks at me like I am crazy.
I have tried the "sit" command and he just looks at me like I am crazy.
I have tried the "sit" command and he just looks at me like I am crazy.
I have tried the "sit" command and he just looks at me like I am crazy.
I have tried the "sit" command and he just looks at me like I am crazy.
I have tried the "sit" command and he just looks at me like I am crazy.
I have tried the "sit" c

In [28]:
instruct_model = Lamini(model_name = "meta-llama/Llama-2-7b-chat-hf")
instruct_output = instruct_model.generate("Tell me how to train my dog to sit")
sentences = re.split(r'(?<=[.!?]) +', instruct_output)
print("\n".join(sentences))

on command.
Training a dog to sit on command is a basic obedience command that can be achieved with patience, consistency, and positive reinforcement.
Here's a step-by-step guide on how to train your dog to sit on command:

1.
Choose a quiet and distraction-free area: Find a quiet area with minimal distractions where your dog can focus on you.
2.
Have treats ready: Choose your dog's favorite treats and have them ready to use as rewards.
3.
Stand in front of your dog: Stand in front of your dog and hold a treat close to their nose.
4.
Move the treat up and back: Slowly move the treat up and back, towards your dog's tail, while saying "sit" in a calm and clear voice.
5.
Dog will sit: As you move the treat, your dog will naturally sit down to follow the treat.
The moment their bottom touches the ground, say "good sit" and give them the treat.
6.
Repeat the process: Repeat steps 3-5 several times, so your dog learns to associate the command "sit" with the action of sitting.
7.
Gradually ph

In [41]:
# chatgpt = Lamini(model_name = "chat-gpt")
# instruct_output_chatgpt = chatgpt.generate("Tell me how to train my dog to sit")
# sentences = re.split(r'(?<=[.!?]) +', instruct_output_chatgpt)
# print("\n".join(sentences))


In [42]:
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/pythia-70m")
model = AutoModelForCausalLM.from_pretrained("EleutherAI/pythia-70m")

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


In [56]:
def inference(text, model, tokenizer, max_input_tokens=1000, max_output_token=100):
    #Tokenize
    input_ids = tokenizer.encode(
        text,
        return_tensors = "pt",
        truncation=True,
        max_length = max_input_tokens
    )
    
    #Generate
    device = model.device
    generated_tokens_with_prompt = model.generate(
        input_ids = input_ids.to(device),
        max_length=max_output_token
    )
    
    #Decode
    generated_text_with_prompt = tokenizer.batch_decode(generated_tokens_with_prompt, skip_special_tokens=True)
    
    #Strip the prompt
    generated_text_answer = generated_text_with_prompt[0][len(text):]
    
    return generated_text_answer

In [57]:
finetuning_dataset_path = "lamini/lamini_docs"
finetuning_dataset = load_dataset(finetuning_dataset_path)
print(finetuning_dataset)

DatasetDict({
    train: Dataset({
        features: ['question', 'answer', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 1260
    })
    test: Dataset({
        features: ['question', 'answer', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 140
    })
})


In [58]:
test_sample = finetuning_dataset["test"][0]
print(test_sample)

print(inference(test_sample["question"], model, tokenizer))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


{'question': 'Can Lamini generate technical documentation or user manuals for software projects?', 'answer': 'Yes, Lamini can generate technical documentation and user manuals for software projects. It uses natural language generation techniques to create clear and concise documentation that is easy to understand for both technical and non-technical users. This can save developers a significant amount of time and effort in creating documentation, allowing them to focus on other aspects of their projects.', 'input_ids': [5804, 418, 4988, 74, 6635, 7681, 10097, 390, 2608, 11595, 84, 323, 3694, 6493, 32, 4374, 13, 418, 4988, 74, 476, 6635, 7681, 10097, 285, 2608, 11595, 84, 323, 3694, 6493, 15, 733, 4648, 3626, 3448, 5978, 5609, 281, 2794, 2590, 285, 44003, 10097, 326, 310, 3477, 281, 2096, 323, 1097, 7681, 285, 1327, 14, 48746, 4212, 15, 831, 476, 5321, 12259, 247, 1534, 2408, 273, 673, 285, 3434, 275, 6153, 10097, 13, 6941, 731, 281, 2770, 327, 643, 7794, 273, 616, 6493, 15], 'attention

Here what we have done is downloaded a model, and dataset and given 1st question in dataset to the notfinetuned model, to see how it 

performs, we can see the nonfinetune model does not performs well, it had learned english word like documents but answer is way off 

what we expected.It had understood its a question answer. 

We can see it doen't understand dataset in terms of knowledge and also doesn't understand what we are expecting form

Lets compare with already finetuned model

In [60]:
instruction_model = AutoModelForCausalLM.from_pretrained("lamini/lamini_docs_finetuned")

In [61]:
print(inference(test_sample["question"],instruction_model, tokenizer))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Yes, Lamini can generate technical documentation or user manuals for software projects. This can be achieved by providing a prompt for a specific technical question or question to the LLM Engine, or by providing a prompt for a specific technical question or question. Additionally, Lamini can be trained on specific technical questions or questions to help users understand the process and provide feedback to the LLM Engine. Additionally, Lamini
